In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Data.VectorSpace
import Data.Basis (Basis)
import Linear(V2(..), ex, ey, _x, _y)
import Data.Semigroup
import qualified Data.Foldable as Hask
import Control.Lens
:opt no-lint
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices, Point(P))

In [3]:
type X = ℝ
type T = ℝ
type U = ℝ
type Ðx'U = ℝ
type Ðt'U = ℝ
type Ðx'Ðt'U = ℝ
type Ðx²'U = ℝ
type x × y = ℝ²
et = ey :: Basis ℝ²

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [4]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' ℝ -> Shade (Colour ℝ)
colourscheme (Shade' u du) = interp (Shade u $ dualNorm du :: Shade ℝ)
 where Just interp = rangeOnGeodesic darkblue orange
 
prettyWebPlot :: PointsWeb ℝ² y -> DynamicPlottable
prettyWebPlot w = plot [ diagramPlot . opacity 0.5 $ fromVertices [P r₁, P r₂]
                       | ((r₁@(V2 x₁ y₁),_),(r₂@(V2 x₂ y₂),_)) <- edg ]
 where edg = webEdges w

In [13]:
colourscheme_heat :: Shade' (U, Ðx'U × Ðt'U) -> Shade (Colour ℝ)
colourscheme_heat = cm . factoriseShade
 where cm :: (Shade' U, Shade' (Ðx'U × Ðt'U)) -> Shade (Colour ℝ)
       cm (Shade' u du, _) = interp (Shade u $ dualNorm du :: Shade ℝ)
       Just interp = rangeOnGeodesic darkblue orange

In [6]:
heatEqn :: DifferentialEqn (X × T) (U, Ðx'U × Ðt'U)
      -- Shade (X×T, U,Ðx'U) -> Shade' (X×T +> (U,Ðx'U)) -> ?(Shade' (X×T +> (U,Ðx'U)))
heatEqn = constLinearPDE
    (arr.LinearFunction $
        \(LinearMap (V2 (ðx'u, V2 ðx'ðx'u ðx'ðt'u)
                        (ðt'u, V2 ðt'ðx'u ðt'ðt'u)))
                  -> (0, V2 ðx'u ðt'u) )
    (arr.LinearFunction $
        \(_u, V2 ðx'u ðt'u)
       -> let ðx'ðx'u = ðt'u
          in LinearMap (V2 (ðx'u, V2 ðx'ðx'u 0)
                           (ðt'u, V2 0 0)) )

In [7]:
--(`linMapAt` V2 0 2) <$>
heatEqn ((V2 0 0,(0,V20)):±[(V2 1 0,(0,0)), (V2 0 1,(0,0)), (V2 0 0,(1,0)), (V2 0 0,(0,1))])
        -- (Shade' (LinearMap $ V2 (0,1) (0,0)) . spanNorm $ Tensor <$>
           --              [V2 (1,0) (0,0), V2 (0,1) (0,0), V2 (0,0) (1,0), V2 (0,0) (0,1)] )

Found hole ‘_’ with type: Shade (X × T, (U, Ðx'U × Ðt'U))
Relevant bindings include it :: LocalDifferentialEqn (X × T) (U, Ðx'U × Ðt'U) (bound at <interactive>:1:1)
In the first argument of ‘heatEqn’, namely ‘_’
In the expression: heatEqn _
In an equation for ‘it’: it = heatEqn _

In [16]:
initState_heat :: X -> (U, Ðx'U × Ðt'U)
initState_heat x = ( u
                   , V2 (- 2 * s*x / cosh (s * (1 - x^2))^2 )
                        (-2 * s / cosh(s * (1 - x^2))^2 - 8 * s^2 * x^2 / cosh(s * (1 - x^2))^2 * u)
                   )
 where u = tanh (s * (1 - x^2))
       s = 0.5

plotWindow $ continFnPlot <$> [ fst . initState_heat
                              , (^._x) . snd . initState_heat
                              , (^._y) . snd . initState_heat]

tf_heat :: Needle X -> Needle T -> PointsWeb (X × T) (Shade' (U, Ðx'U × Ðt'U))
tf_heat δx₀ δt₀ = fromWebNodes euclideanMetric
      $  [(V2 x 0, initState_heat x|±|[(0.1,0), (0,0.1)]) | x<-[-2, δx₀-2 .. 0] ]
      ++ [(V2 x t, zeroV|±|[(1, 0),  (0,1)  ]) | x<-[-2, δx₀-2 .. 0], t<-[δt₀, δt₀*2 .. 1] ]

GraphWindowSpecR2{lBound=-3.4866277099559726, rBound=3.9310593361978943, bBound=-3.52796304107421, tBound=3.7453125189701284, xResolution=640, yResolution=480}

In [17]:
startSt_heat = tf_heat 0.13 0.13
forM_ [ iterateFilterDEqn_static (HighlightInconsistencies $ (-1, 0)|±|[(10, 0),(0, 10)]) heatEqn startSt_heat ]
  $ \tfs ->
    plotWindow
       [ prettyWebPlot $ head tfs
       , plotLatest [ plot (fmap colourscheme_heat tfi)
                       & legendName ("i = "++show i)
                    | (i,tfi) <- zip [0..] tfs ] ]

![Initial state from which to start refining solution of the heat PDE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/PDE-solution-filter/HeatEqn-InitialState.png)

Formulate the heat equation without explicit reference to numerical derivatives
(only _numerically integrating_ the derivatives stored in the nodes):

In [ ]:
type HeatFlow = (U, (Ðx'U × Ðt'U, Ðx'Ðt'U))

μ :: LocalLinear (X × T) HeatFlow +> HeatFlow
μ = 

heatEqn' :: DifferentialEqn (X × T) HeatFlow
      -- Shade (X×T, U,Ðx'U) -> Shade' (X×T +> (U,Ðx'U)) -> ?(Shade' (X×T +> (U,Ðx'U)))
heatEqn' = constLinearDEqn
  (arr.LinearFunction $
        \(_u, (V2 ðx'u ðt'u, ðx'ðt'u))
       -> let ðx'ðx'u = ðt'u
              ðt'ðx'u = ðx'ðt'u
              ðt'ðt'u = 0
              ðx'ðx'ðt'u = 0
              ðt'ðx'ðt'u = 0
                -- ðt'x'u = ðx't'u = ðx'x²'u = ðx²'x'u
       -> LinearMap (V2 (ðx'u, (V2 ðx'ðx'u ðx'ðt'u, ðx'ðx'ðt'u))
                        (ðt'u, (V2 ðt'ðx'u ðt'ðt'u, ðt'ðx'ðt'u))))
  (arr.LinearFunction $
        \(LinearMap (V2 (ðx'u, V2 ðx'ðx'u ðx'ðx'ðt'u)
                        (ðt'u, V2 ðt'ðx'u ðt'ðx'ðt'u)))
            -> (0, (V2 ðx'u ðx'ðx'u)))

In [ ]:
initState_heatFlow :: X -> (U, Ðx'U × Ðt'U)
initState_heatFlow x = ( tanh (s * (1 - x^2))
                       , V2 (- 2 * s*x / cosh (s * (1 - x^2))^2)
                            ((- 8 * s^2 * x^2 * tanh (s * (1 - x^2)) - 2*s) / cosh (s * (1 - x^2))^2) )
 where s = 0.5

plotWindow $ continFnPlot <$> [ fst . initState_heatFlow
                              , (^._x) . snd . initState_heatFlow
                              , (^._y) . snd . initState_heatFlow ]

tf_heatFlow :: Needle X -> Needle T -> PointsWeb (X × T) (Shade' HeatFlow)
tf_heatFlow δx₀ δt₀ = fromWebNodes euclideanMetric
      $  [(V2 x 0, initState_heatFlow x|±|[(0.1,V2 0 0), (0,V2 0.1 0), (0,V2 0 0.1)]) | x<-[xl, xl+δx₀ .. xr] ]
      ++ [(V2 xl t, initState_heatFlow xl|±|[(1,V2 0 0), (0,V2 0.1)]) | t<-tail[0, δt₀, .. 1] ]
      ++ [(V2 x t, zeroV|±|[(1, 0),  (0,1)]) | x<-tail[xl, xl+δx₀ .. xr], t<-tail[0, δt₀ .. 1] ]
 where (xl,xr) = (-2,0)